In [13]:
with open(r"C:\Users\selas\Downloads\taylorswift.txt", 'r') as f:
    text=f.read()

text
tokens=text.encode("utf-8")
tokens=list(tokens)

In [10]:
import regex as re

In [56]:

GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""
class BasicTokenizer():
    def __init__(self, pattern=None):

        self.pattern=GPT4_SPLIT_PATTERN if pattern is None else pattern
        self.compiled_pattern=re.compile(self.pattern)

     

    def get_stats(self, ids, counts=None):
        counts={} if counts==None else counts
        for pair in zip(ids, ids[1:]):

            counts[pair]=counts.get(pair, 0)+1
        return counts


    def merge(self, ids, pair, idx):
        
        new_ids=[]
        i=0
        while i<len(ids):
            if i < len(ids) - 1 and ids[i]==pair[0] and ids[i+1]==pair[1]:
                new_ids.append(idx)
                i+=2
            else:
                new_ids.append(ids[i])
                i+=1
            #print(f"merging{pair}----->{idx}")
        return new_ids

    
    

        
    def train(self, text, vocab_size):
        
        text_chunks = self.compiled_pattern.findall(text)


        # input text preprocessing
        ids = [list(ch.encode("utf-8")) for ch in text_chunks]

 
            
        
        num_merges=vocab_size-256
        merges={}
        vocab={idx: bytes([idx]) for idx in range(256)}
        idx=256
        for i in range(num_merges):
            stats={}
            for chunk_ids in ids:

                self.get_stats(chunk_ids, stats)


            pair=max(stats, key=stats.get)
            ids = [self.merge(chunk_ids, pair, idx) for chunk_ids in ids]
                #print(stats)
                
            ids=self.merge(ids, pair, idx)
            merges[pair]=idx
            '''if i%1000==0:
                print(f"merging({pair}) to {idx})")'''
                #print(len(ids))
            vocab[idx] = vocab[pair[0]] + vocab[pair[1]]
            idx+=1
        
        self.merges=merges
        self.vocab=vocab
        #return(merges)
        print(f"compression ratio: {len(text) / len(ids):.2f}X")

         
        

    def encode_chunk(self, text_bytes):
        ids=list(text_bytes)
        while len(ids)>=2:
            stats=self.get_stats(ids)
            #print(self.merges)
            pair=min(stats, key=lambda p: self.merges.get(p, float('inf')))
            if pair not in self.merges:
                break
            idx=self.merges[pair]
            ids=self.merge(ids, pair, idx)
        return ids
    def encode_ordinary(self, text):
        chuncks=self.compiled_pattern.findall(text)
        ids=[]
        for chunk in chuncks:
            chunk_bytes=chunk.encode('utf-8')
            chunk_ids=self.encode_chunk(chunk_bytes)
            ids.extend(chunk_ids)
        return ids



        
    def decode(self, ids):
        tokens=b"".join(self.vocab[idx] for idx in ids)
        text=tokens.decode("utf-8", errors='replace')
        return text
            



In [65]:

tokin=BasicTokenizer()
tokin.train(text, 500)


compression ratio: 4.01X


In [68]:
print(tokin.decode(tokin.encode_ordinary("ordinary bro")))

ordinary bro
